In [1]:
import prettytable
import tensorflow as tf
from keras import callbacks, initializers, optimizers, regularizers
from keras.layers import (LSTM, Bidirectional, Concatenate, Conv1D, Dense,
                          Dropout, Flatten, Input, MaxPooling1D,
                          TimeDistributed)
from keras.models import Model, Sequential
from sklearn.metrics import (accuracy_score, f1_score, hamming_loss,
                             precision_recall_curve, precision_score,
                             recall_score)
from sklearn.model_selection import train_test_split
from keras.models import load_model


Using TensorFlow backend.


In [2]:
tf.__version__

'2.1.0'

In [3]:
import pandas as pd
import numpy as np
import datetime

In [4]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [5]:
# Loading Tensorboard Extension
%load_ext tensorboard

In [6]:
from MultiChannelBiLSTMCNN import BalanceNet, time_logger_save


In [7]:
# Metrics Calculator Function
def evaluate_model(real, predicted):
    accuracy = accuracy_score(real, predicted)
    hamLoss = hamming_loss(real, predicted)
    # element wise correctness
    term_wise_accuracy = np.sum(np.logical_not(
        np.logical_xor(real, predicted)))/real.size

    macro_precision = precision_score(real, predicted, average='macro')
    macro_recall = recall_score(real, predicted, average='macro')
    macro_f1 = f1_score(real, predicted, average='macro')

    micro_precision = precision_score(real, predicted, average='micro')
    micro_recall = recall_score(real, predicted, average='micro')
    micro_f1 = f1_score(real, predicted, average='micro')

    metricTable = prettytable.PrettyTable()
    metricTable.field_names = ["Metric", "Macro Value", "Micro Value"]
    metricTable.add_row(["Hamming Loss", "{0:.3f}".format(hamLoss), ""])
    metricTable.add_row(
        ["Term Wise Accuracy", "{0:.3f}".format(term_wise_accuracy), ""])

    metricTable.add_row(["Accuracy", "{0:.3f}".format(accuracy), ""])
    metricTable.add_row(["Precision", "{0:.3f}".format(
        macro_precision), "{0:.3f}".format(micro_precision)])
    metricTable.add_row(["Recall", "{0:.3f}".format(
        macro_recall), "{0:.3f}".format(micro_recall)])
    metricTable.add_row(
        ["F1-measure", "{0:.3f}".format(macro_f1), "{0:.3f}".format(micro_f1)])

    print(metricTable)

In [8]:
#Helper Functions
def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr
  
def initial_boost(epoch):
    if epoch==0: return float(8.0)
    elif epoch==1: return float(4.0)
    elif epoch==2: return float(2.0)
    elif epoch==3: return float(1.5)
    else: return float(1.0)

def step_cyclic(epoch):
    try:
        l_r, decay = 1.0, 0.0001
        if epoch%33==0:multiplier = 10
        else:multiplier = 1
        rate = float(multiplier * l_r * 1/(1 + decay * epoch))
        #print("Epoch",epoch+1,"- learning_rate",rate)
        return rate
    except Exception as e:
        print("Error in lr_schedule:",str(e))
        return float(1.0)


Loading Data

In [9]:
mpstDF= pd.read_csv("mpst.csv")
mpstDF

,imdb_id,title,plot_synopsis,tags,split,synopsis_source
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,imdb
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,imdb
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,imdb
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train,imdb
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,imdb
...,...,...,...,...,...,...
14823,tt0219952,Lucky Numbers,"In 1988 Russ Richards (John Travolta), the wea...","comedy, murder",test,wikipedia
14824,tt1371159,Iron Man 2,"In Russia, the media covers Tony Stark's discl...","good versus evil, violence",train,wikipedia
14825,tt0063443,Play Dirty,During the North African Campaign in World War...,anti war,train,wikipedia
14826,tt0039464,High Wall,Steven Kenet catches his unfaithful wife in th...,murder,test,wikipedia


In [10]:
# Data Split Function
def get_partition_Embeddings(x_t1,x_t2,y,df,partition_nm):
    _df=df[df["split"]==partition_nm]
    index_list=list(_df.index)
    temp_array_x_t1=[]
    temp_array_x_t2=[]
    temp_array_y=[]
    for index in index_list:
        temp_array_x_t1.append(x_t1[index,:])
        temp_array_x_t2.append(x_t2[index,:])
        temp_array_y.append(y[index,:])
    temp_array_x_t1=np.array(temp_array_x_t1)
    temp_array_x_t2=np.array(temp_array_x_t2)
    temp_array_y=np.array(temp_array_y)
    return temp_array_x_t1,temp_array_x_t2, temp_array_y

In [11]:
# LOADING BERT  EMBEDDINGS
bert_embedding=np.load(r"D:\CodeRepo\Thesis\Thesis\XLNet\xl_embeddings.npz")
# LOADING XLnet EMBEDDINGS
xl_embedding=np.load(r"D:\CodeRepo\Thesis\Thesis\XLNet\xl_embeddings.npz")
# LOADING LABELS Y
label_values=np.load(r"D:\CodeRepo\Thesis\Thesis\EDA\Y.npz")

In [12]:
# BERT EMBEDDINGS T1 & T2
type1_BERT_Embeddings=bert_embedding["t1"]
type2_BERT_Embeddings=bert_embedding["t2"]
# XLNet EMBEDDINGS T1 & T2
type1_XL_Embeddings=xl_embedding["t1"]
type2_XL_Embeddings=xl_embedding["t2"]
# LABLES Y
label_values=label_values["arr_0"]

In [13]:
# BERT

# FOR TRAIN
type1_BERT_Embeddings_Train,type2_BERT_Embeddings_Train,BERT_label_values_Train=get_partition_Embeddings(type1_BERT_Embeddings,type2_BERT_Embeddings,label_values,mpstDF,"train")
# FOR VALIDATION
type1_BERT_Embeddings_Val,type2_BERT_Embeddings_Val,BERT_label_values_Val=get_partition_Embeddings(type1_BERT_Embeddings,type2_BERT_Embeddings,label_values,mpstDF,"val")
# FOR TEST
type1_BERT_Embeddings_Test,type2_BERT_Embeddings_Test,BERT_label_values_Test=get_partition_Embeddings(type1_BERT_Embeddings,type2_BERT_Embeddings,label_values,mpstDF,"test")

In [14]:
# XLNET

# FOR TRAIN
type1_XL_Embeddings_Train,type2_XL_Embeddings_Train,XLNET_label_values_Train=get_partition_Embeddings(type1_XL_Embeddings,type2_XL_Embeddings,label_values,mpstDF,"train")
# FOR VALIDATION
type1_XL_Embeddings_Val,type2_XL_Embeddings_Val,XLNET_label_values_Val=get_partition_Embeddings(type1_XL_Embeddings,type2_XL_Embeddings,label_values,mpstDF,"val")
# FOR TEST
type1_XL_Embeddings_Test,type2_XL_Embeddings_Test,XLNET_label_values_Test=get_partition_Embeddings(type1_XL_Embeddings,type2_XL_Embeddings,label_values,mpstDF,"test")

HAVING A LOOK AT THE SHAPES OF EACH PARTITION CREATED:

In [15]:
print("SHAPES OF EACH PARTITION _ BERT\n")
print("type1_BERT_Embeddings_Train.shape: ", type1_BERT_Embeddings_Train.shape)
print("type2_BERT_Embeddings_Train.shape: ", type2_BERT_Embeddings_Train.shape)
print("BERT_label_values_Train.shape: "    , BERT_label_values_Train.shape)

print("type1_BERT_Embeddings_Val.shape: ", type1_BERT_Embeddings_Val.shape)
print("type2_BERT_Embeddings_Val.shape: ", type2_BERT_Embeddings_Val.shape)
print("BERT_label_values_Val.shape: "    , BERT_label_values_Val.shape)

print("type1_BERT_Embeddings_Test.shape: ", type1_BERT_Embeddings_Test.shape)
print("type2_BERT_Embeddings_Test.shape: ", type2_BERT_Embeddings_Test.shape)
print("BERT_label_values_Test.shape: "    , BERT_label_values_Test.shape)

print("SHAPES OF EACH PARTITION _ XLNET\n")

print("type1_XL_Embeddings_Train.shape: ", type1_XL_Embeddings_Train.shape)
print("type2_XL_Embeddings_Train.shape: ", type2_XL_Embeddings_Train.shape)
print("XLNET_label_values_Train.shape: " , XLNET_label_values_Train.shape)

print("type1_XL_Embeddings_Val.shape: ", type1_XL_Embeddings_Val.shape)
print("type2_XL_Embeddings_Val.shape: ", type2_XL_Embeddings_Val.shape)
print("XLNET_label_values_Val.shape: " , XLNET_label_values_Val.shape)

print("type1_XL_Embeddings_Test.shape: ", type1_XL_Embeddings_Test.shape)
print("type2_XL_Embeddings_Test.shape: ", type2_XL_Embeddings_Test.shape)
print("XLNET_label_values_Test.shape: " , XLNET_label_values_Test.shape)

print("SHAPES OF EACH PARTITION _ BERT\n")
print("type1_BERT_Embeddings_Train.: ", type1_BERT_Embeddings_Train)
print("type2_BERT_Embeddings_Train.: ", type2_BERT_Embeddings_Train)
print("BERT_label_values_Train.: "    , BERT_label_values_Train)

print("type1_BERT_Embeddings_Val.: ", type1_BERT_Embeddings_Val)
print("type2_BERT_Embeddings_Val.: ", type2_BERT_Embeddings_Val)
print("BERT_label_values_Val.: "    , BERT_label_values_Val)

print("type1_BERT_Embeddings_Test.shape: ", type1_BERT_Embeddings_Test)
print("type2_BERT_Embeddings_Test.shape: ", type2_BERT_Embeddings_Test)
print("BERT_label_values_Test.shape: "    , BERT_label_values_Test)

print("SHAPES OF EACH PARTITION _ XLNET\n")

print("type1_XL_Embeddings_Train: ", type1_XL_Embeddings_Train)
print("type2_XL_Embeddings_Train: ", type2_XL_Embeddings_Train)
print("XLNET_label_values_Train: " , XLNET_label_values_Train)

print("type1_XL_Embeddings_Val: ", type1_XL_Embeddings_Val)
print("type2_XL_Embeddings_Val: ", type2_XL_Embeddings_Val)
print("XLNET_label_values_Val: " , XLNET_label_values_Val)

print("type1_XL_Embeddings_Test: ", type1_XL_Embeddings_Test)
print("type2_XL_Embeddings_Test: ", type2_XL_Embeddings_Test)
print("XLNET_label_values_Test: " , XLNET_label_values_Test)


SHAPES OF EACH PARTITION _ BERT

type1_BERT_Embeddings_Train.shape:  (9489, 768)
type2_BERT_Embeddings_Train.shape:  (9489, 768)
BERT_label_values_Train.shape:  (9489, 71)
type1_BERT_Embeddings_Val.shape:  (2373, 768)
type2_BERT_Embeddings_Val.shape:  (2373, 768)
BERT_label_values_Val.shape:  (2373, 71)
type1_BERT_Embeddings_Test.shape:  (2966, 768)
type2_BERT_Embeddings_Test.shape:  (2966, 768)
BERT_label_values_Test.shape:  (2966, 71)
SHAPES OF EACH PARTITION _ XLNET

type1_XL_Embeddings_Train.shape:  (9489, 768)
type2_XL_Embeddings_Train.shape:  (9489, 768)
XLNET_label_values_Train.shape:  (9489, 71)
type1_XL_Embeddings_Val.shape:  (2373, 768)
type2_XL_Embeddings_Val.shape:  (2373, 768)
XLNET_label_values_Val.shape:  (2373, 71)
type1_XL_Embeddings_Test.shape:  (2966, 768)
type2_XL_Embeddings_Test.shape:  (2966, 768)
XLNET_label_values_Test.shape:  (2966, 71)
SHAPES OF EACH PARTITION _ BERT

type1_BERT_Embeddings_Train.:  [[-2.333117    0.7328038   0.04426328 ... -0.08736744 -0.77967

In [16]:
# function to reshape my input
def tensor_reshape(input, timestep, features):
  reshaped= input.reshape(type1_BERT_Embeddings_Train.shape[0], timestep, features)
  return reshaped

STARTING THE MODEL
possible combination in inputs:
(timestep, features)
- (1, 768)
- (2, 384)
- (3, 256)
- (4, 192)
- (6, 128)
- (12, 64)

## MODEL (3, 256):


In [17]:
INPUT_SHAPE =(3, 256)
EM_L_F_UNITS= 256
EM_L_T_UNITS= 256
# LEFT CHANNEL
LSTM_1F_UNITS= 256
LSTM_1T_UNITS= 256

CONV_2_FILTER= 64
CONV_2_KERNEL= 2
CONV_3_FILTER= 64
CONV_3_KERNEL= 3
CONV_5_FILTER= 64
CONV_5_KERNEL= 4
CONV_6_FILTER= 64
CONV_6_KERNEL= 5
CONV_8_FILTER= 64
CONV_8_KERNEL= 6

# RIGHT CHANNEL 
CONV_4F_FILTERS = 64
CONV_4F_KERNEL = 3
CONV_4T_FILTERS = 64
CONV_4T_KERNEL = 3

CONV_3F_FILTERS = 64
CONV_3F_KERNEL = 3
CONV_3T_FILTERS = 64
CONV_3T_KERNEL = 3

CONV_2F_FILTERS = 64
CONV_2F_KERNEL = 3
CONV_2T_FILTERS = 64
CONV_2T_KERNEL = 3

LSTM_2_C_L_UNITS = 32

OUTPUT_DENSE_UNIT =128
OUTPUT_SIZE =71



In [18]:
optimizer_list = ['adam']
dataset_X={
    "bert_t1":[
        type1_BERT_Embeddings_Train,
        type1_BERT_Embeddings_Val,
        type1_BERT_Embeddings_Test
        ],
    "bert_t2":[
        type2_BERT_Embeddings_Train,
        type2_BERT_Embeddings_Val,
        type2_BERT_Embeddings_Test
        ],
    "xlnet_t1":[
        type1_XL_Embeddings_Train,
        type1_XL_Embeddings_Val,
        type1_XL_Embeddings_Test
        ],
    "xlnet_t2":[
        type2_XL_Embeddings_Train,
        type2_XL_Embeddings_Val,
        type2_XL_Embeddings_Test
    ]
}
dataset_Y ={
    "bert":[
        BERT_label_values_Train,
        BERT_label_values_Val,
        BERT_label_values_Test
        ],
    "xlnet":[
        XLNET_label_values_Train,
        XLNET_label_values_Val,
        XLNET_label_values_Test
        ]
}

inp_shape_str = "3_256"


In [19]:
model_dict ={}
t_list =[x/10 for x in range(1,10)]
time_dict=[]

for ds in dataset_X.keys():
    print("\nFOR DATASET: ", ds)
    for opt in optimizer_list:
        print("FOR OPTIMIZER: ",opt)
        key= "model_"+inp_shape_str+"_"+opt+"_"+ds
        model_type = ds+"_"+opt
        print("\n###############\nKEY= "+key+"\n#############")
        print("\n###############\nmodel_type= "+model_type+"\n#############")
        model_dict[key] = BalanceNet(INPUT_SHAPE,
                                    EM_L_F_UNITS,
                                    EM_L_T_UNITS,
                                    LSTM_1F_UNITS,
                                    LSTM_1T_UNITS,
                                    CONV_2_FILTER,
                                    CONV_2_KERNEL,
                                    CONV_3_FILTER,
                                    CONV_3_KERNEL,
                                    CONV_5_FILTER,
                                    CONV_5_KERNEL,
                                    CONV_6_FILTER,
                                    CONV_6_KERNEL,
                                    CONV_8_FILTER,
                                    CONV_8_KERNEL,
                                    CONV_4F_FILTERS,
                                    CONV_4F_KERNEL,
                                    CONV_4T_FILTERS,
                                    CONV_4T_KERNEL,
                                    CONV_3F_FILTERS,
                                    CONV_3F_KERNEL,
                                    CONV_3T_FILTERS,
                                    CONV_3T_KERNEL,
                                    CONV_2F_FILTERS,
                                    CONV_2F_KERNEL,
                                    CONV_2T_FILTERS,
                                    CONV_2T_KERNEL,
                                    LSTM_2_C_L_UNITS,
                                    OUTPUT_DENSE_UNIT,
                                    OUTPUT_SIZE,
                                    optimizer_name= opt)
        
        if 'bert' in ds:
            em_type='bert'
        if 'xlnet' in ds:
            em_type='xlnet'
        model_dict[key].fitModel(train_x =dataset_X[ds][0], 
                                            train_y=dataset_Y[em_type][0], 
                                            val_x= dataset_X[ds][1], 
                                            val_y = dataset_Y[em_type][1], 
                                            model_type = model_type, 
                                            epochs_count = 20 , 
                                            batch_count = 64,  time_dict=time_dict, key=key)
        model_dict[key].predictModel(test_x=dataset_X[ds][2], test_y=dataset_Y[em_type][2], threshold_list=t_list,  time_dict=time_dict, key=key)
        
time_logger_save(time_dict,inp_shape_str+"_time")



FOR DATASET:  bert_t1
FOR OPTIMIZER:  adam

###############
KEY= model_3_256_adam_bert_t1
#############

###############
model_type= bert_t1_adam
#############
inp_layer (None, 3, 256)
embedding_layer_frozen (None, 3, 256)
embedding_layer_train (None, 3, 256)
l_lstm_1f (None, 3, 512)
l_lstm_1t (None, 3, 512)
l_lstm1 (None, 6, 512)
l_conv_2 (None, 5, 64)
l_conv_3 (None, 4, 64)
l_conv_5 (None, 3, 64)
l_conv_6 (None, 2, 64)
l_conv_8 (None, 1, 64)
l_lstm_c (None, 15, 64)
l_conv_4f (None, 1, 64)
l_conv_4t (None, 1, 64)
l_conv_3f (None, 1, 64)
l_conv_3t (None, 1, 64)
l_conv_2f (None, 1, 64)
l_conv_2t (None, 1, 64)
l_merge_2 (None, 6, 64)
l_c_lstm (None, 6, 64)
Training Progress for: bert_t1_adam
Reshaping inputs
Reshaped Into Shape:  (9489, 3, 256)
Reshaped Into Shape:  (2373, 3, 256)
Reshaped Successfully!
Train on 9489 samples, validate on 2373 samples
Epoch 1/20
9489/9489 [==============================] - 11s 1ms/step - loss: 10.6528 - acc: 0.0424 - categorical_accuracy: 0.0424 - top_k_

TypeError: fitModel() missing 2 required positional arguments: 'time_dict' and 'key'

# END

In [ ]:
model_dict={}
model_dict["model_1_768_adam_bert_t1"]= BalanceNet(INPUT_SHAPE,
                                                    EM_L_F_UNITS,
                                                    EM_L_T_UNITS,
                                                    LSTM_1F_UNITS,
                                                    LSTM_1T_UNITS,
                                                    CONV_2_FILTER,
                                                    CONV_2_KERNEL,
                                                    CONV_3_FILTER,
                                                    CONV_3_KERNEL,
                                                    CONV_5_FILTER,
                                                    CONV_5_KERNEL,
                                                    CONV_6_FILTER,
                                                    CONV_6_KERNEL,
                                                    CONV_8_FILTER,
                                                    CONV_8_KERNEL,
                                                    CONV_4F_FILTERS,
                                                    CONV_4F_KERNEL,
                                                    CONV_4T_FILTERS,
                                                    CONV_4T_KERNEL,
                                                    CONV_3F_FILTERS,
                                                    CONV_3F_KERNEL,
                                                    CONV_3T_FILTERS,
                                                    CONV_3T_KERNEL,
                                                    CONV_2F_FILTERS,
                                                    CONV_2F_KERNEL,
                                                    CONV_2T_FILTERS,
                                                    CONV_2T_KERNEL,
                                                    LSTM_2_C_L_UNITS,
                                                    OUTPUT_DENSE_UNIT,
                                                    OUTPUT_SIZE,
                                                    optimizer_name= 'adam')

In [ ]:
model_dict["model_1_768_adam_bert_t1"].fitModel(train_x =dataset_X["bert_t1"][0], 
                                                train_y=dataset_Y['bert'][0], 
                                                val_x= dataset_X["bert_t1"][1], 
                                                val_y = dataset_Y['bert'][1], 
                                                model_type = "bert_t1_adam", 
                                                epochs_count = 20 , 
                                                batch_count = 64)